# Convolutional Neural Network

### Importing the libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from keras.preprocessing import image
from tensorflow.keras.models import load_model

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
tf.__version__

'2.18.0'

## Part 1 - Data Preprocessing

In [5]:
batch_size = 8
img_height = 300
img_width = 300
image_size = (img_height, img_width)

### Processando o TrainSet

In [6]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'dataset/',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size,)

Found 2615 files belonging to 3 classes.
Using 2092 files for training.


### Processando o TestSet

In [7]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    'dataset/',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size,)

Found 2615 files belonging to 3 classes.
Using 2092 files for training.


#### Primeiras 9 imagens

In [11]:
class_names = training_set.class_names
print(class_names)

['Cercosporiose', 'Ferrugem', 'Saudavel']


In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in training_set.take(1):
  for i in range(8):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

## Configurando o conjunto de dados

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

training_set = training_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_set = validation_set.cache().prefetch(buffer_size=AUTOTUNE)

## Padronizando os dados

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = training_set.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

# Agora os valores dos pixels estão entre 0 e 1
print(np.min(first_image), np.max(first_image))

## Construíndo a CNN

### Iniciando o modelo

In [ ]:
num_classes = len(class_names)

cnn = Sequential(
    [
        layers.Rescaling(1./255, 
                         input_shape=(img_height, 
                                      img_width, 
                                      3)),
        layers.RandomFlip("horizontal",
                          input_shape=(img_height,
                                       img_width,
                                       3)),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        
        layers.Conv2D(16, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(256, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(512, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes)
    ]
)

### Compiling the CNN

In [ ]:
cnn.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Sumario do modelo

In [ ]:
cnn.summary()

### Treinando a CNN

In [ ]:
epochs = 20

history = cnn.fit(
    training_set, 
    validation_data = validation_set, 
    epochs = epochs)

### Visualizando os dados do treinamento

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Precisão do Treinamento")
plt.plot(epochs_range, val_acc, label="Precisão da Validação")
plt.legend(loc='lower right')
plt.title('Precisão do treinamento e da validação')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Precisão do Treinamento")
plt.plot(epochs_range, val_loss, label="Precisão da Validação")
plt.legend(loc='lower right')
plt.title('Perda no treinamento e na validação')

plt.show()

## Fazendo apenas uma predição

In [12]:
test_image = image.load_img('./testset/Cercosporiose/cercosporiose15.jpg', target_size = image_size)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

print(class_names)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
['Cercosporiose', 'Ferrugem', 'Saudavel']


In [23]:
print(result)

[[  4.3769207    0.13541324 -21.533245  ]]


In [26]:
def getProbability(result=result):
    probabilities = np.exp(result) / np.sum(np.exp(result), axis=1, keepdims=True)    

    predicted_index = np.argmax(probabilities)
    
    predicted_class_name = class_names[predicted_index]

    return predicted_class_name

print(f"A classe prevista é {getProbability()}")

A classe prevista é Cercosporiose


### Salvando o modelo

In [ ]:
cnn.save("SicknessMinder_V3_4_1.keras")

### Abrindo o modelo

In [9]:
cnn = load_model("../Modelos/SicknessMinder_V3_4_1.keras")
loss, accuracy = cnn.evaluate(validation_set)
print(f'Loss: {loss}, Accuracy: {accuracy}')

262/262 ━━━━━━━━━━━━━━━━━━━━ 17s 63ms/step - accuracy: 0.9378 - loss: 0.1641
Loss: 0.1658826321363449, Accuracy: 0.9364244937896729
